In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from scipy.integrate import cumtrapz
from scipy.stats import gaussian_kde
import scipy.stats as stats
from getdist import plots, MCSamples
import matplotlib as mpl
from astropy.cosmology import Planck15
from scipy.interpolate import InterpolatedUnivariateSpline
import fsps
import emcee
from scipy.special import hyp2f1

import tensorflow as tf
import tensorflow_probability as tfp
import tqdm
from tqdm import trange
tfb = tfp.bijectors
tfd = tfp.distributions
tfkl = tf.keras.layers
tfpl = tfp.layers
tfk = tf.keras

import sys
sys.path.append('/Users/justinalsing/Documents/science/steppz/code')
from plotting import triangle_plot
from utils import *
from priors import *
from affine import *
from ndes import *

/Users/justinalsing/.pyenv/versions/3.8.6/envs/base3.8/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Import relevant models

In [2]:
# import the relevant models
log10sSFR_emulator = RegressionNetwork(restore=True, restore_filename='ProspectorAlpha_log10sSFR_emulator.pkl')
baseline_SFR_prior_log_prob = RegressionNetwork(restore=True, restore_filename='ProspectorAlpha_baseline_SFR_prior_logprob.pkl')

# set up the prior class
Prior = ProspectorAlphaBaselinePrior(baselineSFRprior=baseline_SFR_prior_log_prob, 
                             log10sSFRemulator=log10sSFR_emulator, 
                             log10sSFRprior=log10sSFRpriorMizuki, 
                             log10sSFRuniformlimits=tfd.Uniform(low=-14, high=-8), 
                             redshift_prior=redshift_volume_prior)

In [3]:
# initialize walkers for sampling
n_walkers = 2000
n_steps = 1000

# baseline prior draws
bijector = tfb.Blockwise([tfb.Invert(tfb.Chain([tfb.Invert(tfb.NormalCDF()), tfb.Scale(1./(Prior.upper[_]-Prior.lower[_])), tfb.Shift(-Prior.lower[_])])) for _ in range(Prior.n_sps_parameters)])
baseline_draws = bijector(Prior.baselinePrior.sample((30000, Prior.n_sps_parameters)))

# reject those outside SFR prior range
sfh = tf.gather(baseline_draws, [2, 3, 4, 5, 6, 7, 1, 14], axis=-1)
log10sSFR = tf.squeeze(log10sSFR_emulator(sfh))
baseline_draws = tf.squeeze(tf.gather(baseline_draws, indices=tf.where((log10sSFR > -14) & (log10sSFR < -8)), axis=0), axis=1)

# convert log10M to N
baseline_draws = baseline_draws.numpy()
baseline_draws[...,0] = -2.5*baseline_draws[...,0] + distance_modulus(tf.math.maximum(1e-5, baseline_draws[...,-1]))
log_prior = Prior.log_prob(baseline_draws).numpy()
baseline_draws = baseline_draws[~np.isinf(log_prior),:]
baseline_draws = tf.convert_to_tensor(baseline_draws)

# current state
current_state = [baseline_draws[0:n_walkers,:], baseline_draws[n_walkers:2*n_walkers,:]]

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [4]:
n_batches = 64
n_samples = 100000

# burn in
chain = affine_sample(Prior.log_prob, 1000, current_state)
current_state = [chain[-1,0:n_walkers,:], chain[-1,n_walkers:,:]]

for batch in range(n_batches):

    chain = affine_sample(Prior.log_prob, 25, current_state)
    current_state = [chain[-1,0:n_walkers,:], chain[-1,n_walkers:,:]]
    parameters = Prior.bijector(chain).numpy().reshape((25*2*n_walkers, 15))
    
    # save the parameters
    np.save('../model_Prospector-alpha/training_data_prior/parameters/parameters{}.npy'.format(batch), parameters)

100%|██████████| 24/24 [00:04<00:00,  5.80it/s]
